# Data Source Dashboard

In [ ]:
# ========== Bootstrap: ensure pandas is available =========
import importlib, subprocess, sys

def _ensure(pkg_name):
    try:
        importlib.import_module(pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[pkg_name] = importlib.import_module(pkg_name)

_ensure('pandas')
from pathlib import Path
import pandas as pd

catalog = pd.read_csv(Path('..', '..', 'data', 'catalog.csv'))
catalog['path'] = catalog.apply(lambda r: str(Path('data', r['category'], r['source'], r['folder'])), axis=1)
dashboard = catalog[['path', 'last_fetched']].sort_values('path').reset_index(drop=True)
dashboard

In [ ]:
headlines_path = Path("analysis", "headlines", "latest.csv")\nheadline_count = sum(1 for _ in open(headlines_path)) - 1\nprint(f"{headline_count} headlines in {headlines_path}")